<a href="https://colab.research.google.com/github/01PrathamS/MahaTTS_/blob/main/notebooks/experiment_vllm_transfer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/dubverse-ai/MahaTTS.git
!wget https://huggingface.co/Dubverse/MahaTTS/resolve/main/maha_tts/pretrained_models/Smolie-en/t2s_best.pt
#!wget https://huggingface.co/Dubverse/MahaTTS/resolve/main/maha_tts/pretrained_models/Smolie-in/t2s_best.pt

Cloning into 'MahaTTS'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 218 (delta 27), reused 12 (delta 10), pack-reused 180 (from 1)
Receiving objects: 100% (218/218), 97.21 KiB | 759.00 KiB/s, done.
Resolving deltas: 100% (121/121), done.
--2025-04-21 10:17:35--  https://huggingface.co/Dubverse/MahaTTS/resolve/main/maha_tts/pretrained_models/Smolie-en/t2s_best.pt
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.118, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e4/c1/e4c10f18697efa05f94cbc9159cc5d02d7e878b2a51adfde09eab8362e513d8b/6be1b489366ebbd35e55404be875804d380b0430587319f67e592da7ba1b5240?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27t2s_best.pt%3B+filename%3D%22t2s_best.pt%22%3B&Expires=17452342

In [ ]:
%cd MahaTTS/

/content/MahaTTS


In [ ]:
pwd

'/content/MahaTTS'

In [ ]:
import torch
from maha_tts.models.autoregressive import TS_model  # Adjust path if needed

def check_state_dict_keys(checkpoint_path):
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    ckpt_keys = set(checkpoint['state_dict'].keys())

    # Initialize a fresh model with the same config
    model = TS_model()
    model_keys = set(model.state_dict().keys())

    # Compare keys
    missing_keys = model_keys - ckpt_keys
    extra_keys = ckpt_keys - model_keys

    if missing_keys:
        print("❌ Missing keys in checkpoint:")
        for key in missing_keys:
            print("  ", key)
    else:
        print("✅ No missing keys.")

    if extra_keys:
        print("⚠️ Extra keys in checkpoint:")
        for key in extra_keys:
            print("  ", key)
    else:
        print("✅ No extra keys.")

    return not missing_keys and not extra_keys

# Example usage
if __name__ == '__main__':

    check_state_dict_keys("/content/t2s_best.pt")


❌ Missing keys in checkpoint:
   language_embed.weight
✅ No extra keys.


In [ ]:
!pip install -r requirements.txt

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel

# Small custom GPT2 config
hf_config = GPT2Config(
    n_embd=16,
    n_layer=1,
    n_head=1,
    n_positions=32,
    vocab_size=100
)

hf_model = GPT2LMHeadModel(hf_config)
hf_state_dict = hf_model.state_dict()

print("🤗 Hugging Face GPT-2 Weights:\n")
for k, v in hf_state_dict.items():
    print(f"{k}: {v.shape}")


🤗 Hugging Face GPT-2 Weights:

transformer.wte.weight: torch.Size([100, 16])
transformer.wpe.weight: torch.Size([32, 16])
transformer.h.0.ln_1.weight: torch.Size([16])
transformer.h.0.ln_1.bias: torch.Size([16])
transformer.h.0.attn.c_attn.weight: torch.Size([16, 48])
transformer.h.0.attn.c_attn.bias: torch.Size([48])
transformer.h.0.attn.c_proj.weight: torch.Size([16, 16])
transformer.h.0.attn.c_proj.bias: torch.Size([16])
transformer.h.0.ln_2.weight: torch.Size([16])
transformer.h.0.ln_2.bias: torch.Size([16])
transformer.h.0.mlp.c_fc.weight: torch.Size([16, 64])
transformer.h.0.mlp.c_fc.bias: torch.Size([64])
transformer.h.0.mlp.c_proj.weight: torch.Size([64, 16])
transformer.h.0.mlp.c_proj.bias: torch.Size([16])
transformer.ln_f.weight: torch.Size([16])
transformer.ln_f.bias: torch.Size([16])
lm_head.weight: torch.Size([100, 16])


In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(model="gpt2", device="cuda")

llm


INFO 04-21 10:03:09 [__init__.py:239] Automatically detected platform cuda.
INFO 04-21 10:03:17 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 04-21 10:03:31 [config.py:689] This model supports multiple tasks: {'score', 'reward', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
WARNING 04-21 10:03:31 [arg_utils.py:1731] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-21 10:03:31 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='gpt2', speculative_config=None, tokenizer='gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingC

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 10:03:35 [loader.py:458] Loading weights took 0.33 seconds
INFO 04-21 10:03:36 [model_runner.py:1146] Model loading took 0.2378 GiB and 1.685458 seconds
INFO 04-21 10:03:37 [worker.py:267] Memory profiling takes 0.69 seconds
INFO 04-21 10:03:37 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 04-21 10:03:37 [worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 04-21 10:03:37 [executor_base.py:112] # cuda blocks: 22815, # CPU blocks: 7281
INFO 04-21 10:03:37 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 356.48x
INFO 04-21 10:03:41 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 04-21 10:04:10 [model_runner.py:1598] Graph capturing finished in 29 secs, took 0.14 GiB
INFO 04-21 10:04:10 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 34.59 seconds


In [ ]:

for name, param in llm.model.named_parameters():
    print(name, param.shape)


AttributeError: 'LLM' object has no attribute 'model'

In [ ]:
dir(llm)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load GPT-2 base model
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Overwrite weights with dummy values (e.g., all zeros)
for name, param in model.named_parameters():
    param.data.fill_(0.0)

# Save to disk
model.save_pretrained("dummy_gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.save_pretrained("dummy_gpt2")

('dummy_gpt2/tokenizer_config.json',
 'dummy_gpt2/special_tokens_map.json',
 'dummy_gpt2/vocab.json',
 'dummy_gpt2/merges.txt',
 'dummy_gpt2/added_tokens.json',
 'dummy_gpt2/tokenizer.json')

In [ ]:
from vllm import LLM, SamplingParams

# Load the dummy model
llm = LLM(model="dummy_gpt2")

# Try a test prompt
sampling_params = SamplingParams(max_tokens=10)
output = llm.generate("Hello, how are you?", sampling_params=sampling_params)

print(output)


INFO 04-21 10:13:11 [__init__.py:239] Automatically detected platform cuda.
INFO 04-21 10:13:14 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 04-21 10:13:27 [config.py:689] This model supports multiple tasks: {'reward', 'score', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
WARNING 04-21 10:13:27 [arg_utils.py:1731] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-21 10:13:27 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='dummy_gpt2', speculative_config=None, tokenizer='dummy_gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_conf

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 10:13:30 [loader.py:458] Loading weights took 0.32 seconds
INFO 04-21 10:13:31 [model_runner.py:1146] Model loading took 0.2378 GiB and 0.349979 seconds
INFO 04-21 10:13:32 [worker.py:267] Memory profiling takes 0.67 seconds
INFO 04-21 10:13:32 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 04-21 10:13:32 [worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 04-21 10:13:32 [executor_base.py:112] # cuda blocks: 22815, # CPU blocks: 7281
INFO 04-21 10:13:32 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 356.48x
INFO 04-21 10:13:35 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 04-21 10:14:14 [model_runner.py:1598] Graph capturing finished in 39 secs, took 0.14 GiB
INFO 04-21 10:14:14 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 43.47 seconds


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=0, prompt='Hello, how are you?', prompt_token_ids=[15496, 11, 703, 389, 345, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='BA qui titled sequelTal Huaweiryn Moose Titan compass', token_ids=(4339, 45567, 11946, 16304, 31466, 43208, 29441, 44833, 11977, 31855), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1745230454.795379, last_token_time=1745230454.9059412, first_scheduled_time=1745230454.839671, first_token_time=1745230454.8692741, time_in_queue=0.04429197311401367, finished_time=1745230454.9061284, scheduler_time=0.0018100570005117333, model_forward_time=None, model_execute_time=None, spec_token_acceptance_counts=[0]), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [ ]:
checkpoint["state_dict"].keys()

odict_keys(['gpt.h.0.ln_1.weight', 'gpt.h.0.ln_1.bias', 'gpt.h.0.attn.c_attn.weight', 'gpt.h.0.attn.c_attn.bias', 'gpt.h.0.attn.c_proj.weight', 'gpt.h.0.attn.c_proj.bias', 'gpt.h.0.ln_2.weight', 'gpt.h.0.ln_2.bias', 'gpt.h.0.mlp.c_fc.weight', 'gpt.h.0.mlp.c_fc.bias', 'gpt.h.0.mlp.c_proj.weight', 'gpt.h.0.mlp.c_proj.bias', 'gpt.h.1.ln_1.weight', 'gpt.h.1.ln_1.bias', 'gpt.h.1.attn.c_attn.weight', 'gpt.h.1.attn.c_attn.bias', 'gpt.h.1.attn.c_proj.weight', 'gpt.h.1.attn.c_proj.bias', 'gpt.h.1.ln_2.weight', 'gpt.h.1.ln_2.bias', 'gpt.h.1.mlp.c_fc.weight', 'gpt.h.1.mlp.c_fc.bias', 'gpt.h.1.mlp.c_proj.weight', 'gpt.h.1.mlp.c_proj.bias', 'gpt.h.2.ln_1.weight', 'gpt.h.2.ln_1.bias', 'gpt.h.2.attn.c_attn.weight', 'gpt.h.2.attn.c_attn.bias', 'gpt.h.2.attn.c_proj.weight', 'gpt.h.2.attn.c_proj.bias', 'gpt.h.2.ln_2.weight', 'gpt.h.2.ln_2.bias', 'gpt.h.2.mlp.c_fc.weight', 'gpt.h.2.mlp.c_fc.bias', 'gpt.h.2.mlp.c_proj.weight', 'gpt.h.2.mlp.c_proj.bias', 'gpt.h.3.ln_1.weight', 'gpt.h.3.ln_1.bias', 'gpt.h.3

In [ ]:
model.state_dict().keys()

odict_keys(['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.1.attn.c_attn.bias', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.ln_2.weight', 'transformer.h.1.ln_2.bias', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.1.mlp.c_fc.bias', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.2.ln_1.weight', 'transformer.h.2.ln_1.bias', 'transformer.h.2.attn.c_attn.weight', 'transformer.h.2.attn.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams


model = AutoModelForCausalLM.from_pretrained("gpt2")

checkpoint = torch.load("t2s_best.pt")

if "state_dict" in checkpoint:
    state_dict = checkpoint["state_dict"]
else:
    state_dict = checkpoint

model = AutoModelForCausalLM.from_pretrained("gpt2")

model.state_dict().keys()

odict_keys(['transformer.wte.weight', 'transformer.wpe.weight', 'transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.1.ln_1.weight', 'transformer.h.1.ln_1.bias', 'transformer.h.1.attn.c_attn.weight', 'transformer.h.1.attn.c_attn.bias', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.ln_2.weight', 'transformer.h.1.ln_2.bias', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.1.mlp.c_fc.bias', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.2.ln_1.weight', 'transformer.h.2.ln_1.bias', 'transformer.h.2.attn.c_attn.weight', 'transformer.h.2.attn.

In [ ]:
model.load_state_dict(state_dict, strict=False)  # strict=False helps ignore any small mismatches

model.save_pretrained("dummy_gpt2_from_pt")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.save_pretrained("dummy_gpt2_from_pt")


llm = LLM(model="dummy_gpt2_from_pt")

sampling_params = SamplingParams(max_tokens=10)
# output = llm.generate("Hello, how are you?", sampling_params=sampling_params)

# print(output)

model.state_dict().keys()

INFO 04-21 10:37:17 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 04-21 10:37:17 [config.py:689] This model supports multiple tasks: {'embed', 'generate', 'classify', 'score', 'reward'}. Defaulting to 'generate'.
INFO 04-21 10:37:17 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='dummy_gpt2_from_pt', speculative_config=None, tokenizer='dummy_gpt2_from_pt', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 10:37:18 [loader.py:458] Loading weights took 0.32 seconds
INFO 04-21 10:37:18 [model_runner.py:1146] Model loading took 0.2350 GiB and 0.344352 seconds


AssertionError: Error in memory profiling. Initial used memory 15576530944, currently used memory 15576530944. This happens when the GPU memory was not properly cleaned up before initializing the vLLM instance.

In [ ]:

from vllm import LLM, SamplingParams

# Load the dummy model
llm = LLM(model="dummy_gpt2_from_pt")

# Try a test prompt
sampling_params = SamplingParams(max_tokens=10)
output = llm.generate("Hello, how are you?", sampling_params=sampling_params)

print(output)


INFO 04-21 10:20:49 [__init__.py:239] Automatically detected platform cuda.
INFO 04-21 10:20:59 [config.py:2832] Downcasting torch.float32 to torch.float16.
INFO 04-21 10:21:13 [config.py:689] This model supports multiple tasks: {'reward', 'generate', 'score', 'embed', 'classify'}. Defaulting to 'generate'.
WARNING 04-21 10:21:13 [arg_utils.py:1731] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-21 10:21:13 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='dummy_gpt2_from_pt', speculative_config=None, tokenizer='dummy_gpt2_from_pt', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cud

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 10:21:16 [loader.py:458] Loading weights took 0.38 seconds
INFO 04-21 10:21:17 [model_runner.py:1146] Model loading took 0.2378 GiB and 0.433700 seconds
INFO 04-21 10:21:18 [worker.py:267] Memory profiling takes 0.92 seconds
INFO 04-21 10:21:18 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 04-21 10:21:18 [worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 04-21 10:21:19 [executor_base.py:112] # cuda blocks: 22815, # CPU blocks: 7281
INFO 04-21 10:21:19 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 356.48x
INFO 04-21 10:21:22 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 04-21 10:21:53 [model_runner.py:1598] Graph capturing finished in 31 secs, took 0.14 GiB
INFO 04-21 10:21:53 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 35.67 seconds


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=0, prompt='Hello, how are you?', prompt_token_ids=[15496, 11, 703, 389, 345, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=" I'm good. I know I've overestimated", token_ids=(314, 1101, 922, 13, 314, 760, 314, 1053, 41850, 15655), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1745230913.2720954, last_token_time=1745230913.3923564, first_scheduled_time=1745230913.3131213, first_token_time=1745230913.3524654, time_in_queue=0.04102587699890137, finished_time=1745230913.392516, scheduler_time=0.0017275579994020518, model_forward_time=None, model_execute_time=None, spec_token_acceptance_counts=[0]), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [ ]:
!pip install vllm